In [1]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, AutoModel, Trainer, TrainingArguments, BertForMaskedLM, AutoModelForCausalLM, LlamaTokenizer, LlamaForCausalLM
from transformers import AdamW
import torch
from torch.utils.data import DataLoader, TensorDataset
from datasets import Dataset
from langchain_text_splitters import RecursiveCharacterTextSplitter
from tqdm.notebook import tqdm, trange
import numpy as np
from scipy.spatial.distance import cosine
from transformers import GPT2LMHeadModel, GPT2Tokenizer, GenerationConfig
import os
import json
from peft import PeftModel, PeftConfig
from langchain_core.embeddings import Embeddings
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
import pandas as pd
from pathlib import Path

In [3]:
!nvidia-smi

Sun Oct 27 06:20:38 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla V100-SXM3-32GB           Off |   00000000:05:00.0 Off |                    0 |
| N/A   35C    P0             62W /  350W |       1MiB /  32768MiB |      3%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.cuda.empty_cache()

Подключение к базе данных:

In [5]:
DATASET_DIRECTORY_PATH: Path = Path("test_data")
CONVERTED_DATASET_DIRECTORY_PATH: Path = Path("test_data_working")
DATASET_TASKS_FILE_PATH: Path = Path("test.csv")

QDRANT_URL = "http://213.171.5.51:6333"
SIZE = 1024

MODEL_DIRECTORY = "models2"

FULL_COLLECTION_NAME = "GLOBAL"

In [6]:
from converter import convert_dataset

dataset_documents_files_paths = convert_dataset(DATASET_DIRECTORY_PATH,
    output_directory_path=CONVERTED_DATASET_DIRECTORY_PATH,
    converting_suffixes_list=[])

copying Том 2 ПДВ Эко Агро.docx
copying Том 1 Инвентаризация Эко Агро.docx


In [7]:
from text import get_text_blocks

dataset_documents_texts_blocks = dict[str, list[str]]()
dataset_documents_texts_blocks[FULL_COLLECTION_NAME] = []

for dataset_document_file_path in dataset_documents_files_paths:
    dataset_document_text_blocks = get_text_blocks(dataset_document_file_path)

    dataset_document_file_name = dataset_document_file_path.stem
    dataset_documents_texts_blocks[dataset_document_file_name] = \
        dataset_document_text_blocks
    
    dataset_documents_texts_blocks[FULL_COLLECTION_NAME] += \
        dataset_document_text_blocks

In [8]:
class CustomEmbedding(Embeddings):
    def __init__(self, directory: str = "models2"):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model = BertForMaskedLM.from_pretrained(directory)
        self.tokenizer = AutoTokenizer.from_pretrained(directory)
        self.model.to(self.device)

    def embed_query(self, text: str) -> list[float]:
        return extract_features(text, self.model, self.tokenizer).tolist()

    def embed_documents(self, texts: list[str]) -> list[list[float]]:
        return list(map(lambda text: self.embed_query(text), texts))


def extract_features(text, model, tokenizer):
    model.eval()

    text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
        tokenizer,
        chunk_size=512,
        chunk_overlap=64
    )

    chunks = text_splitter.split_text(text)

    features = []

    for chunk in chunks:
        inputs = tokenizer(chunk, return_tensors="pt", truncation=True, padding="max_length", max_length=512).to(device)
        with torch.no_grad():
            outputs = model(**inputs, output_hidden_states=True)
        last_hidden_state = outputs.hidden_states[-1]

        attention_mask = inputs['attention_mask']
        mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size())
        sum_embeddings = torch.sum(last_hidden_state * mask_expanded, 1)
        mean_embeddings = sum_embeddings / torch.clamp(mask_expanded.sum(1), min=1e-9)

        features.append(mean_embeddings)

    return torch.stack(features).mean(dim=0).squeeze(0).cpu()

embedding = CustomEmbedding(MODEL_DIRECTORY)

# получение контекста о запросе
def get_context(question: str, qdrant_collection_name: str, k: int = 15) -> list[str]:
    # просто подключаемся к базе данных ответов
    client = QdrantClient(url=QDRANT_URL)

    if not client.collection_exists(qdrant_collection_name):
        client.create_collection(qdrant_collection_name, vectors_config=VectorParams(size=SIZE, distance=Distance.COSINE))

    embedding = CustomEmbedding()

    qdrant = QdrantVectorStore(
        client=client,
        collection_name=qdrant_collection_name,
        embedding=embedding,
    )

    # достаем k ближайших по смыслу частей и добавляем в контекст на дальнейшую обработку
    return [i.page_content for i in qdrant.similarity_search(question, k=k)]


BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


In [9]:
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
from langchain_qdrant import QdrantVectorStore

qdrant_client = QdrantClient(url=QDRANT_URL)

for collection_name, collection_text_blocks in dataset_documents_texts_blocks.items():
    if not qdrant_client.collection_exists(collection_name):
        print(f"Creating collection \"{collection_name}\"")
        qdrant_client.create_collection(collection_name,
            vectors_config=VectorParams(size=SIZE, distance=Distance.COSINE))
    
        collection_qdrant_vector_store = QdrantVectorStore(
            client=qdrant_client,
            collection_name=collection_name,
            embedding=embedding
        )

        print(f"Uploading text blocks into collection \"{collection_name}\"")
        collection_qdrant_vector_store.add_texts(collection_text_blocks)


In [17]:
LLM_MODEL_NAME = "IlyaGusev/saiga_7b_lora"
DEFAULT_MESSAGE_TEMPLATE = "<s>{role}\n{content}</s>"
DEFAULT_RESPONSE_TEMPLATE = "<s>bot\n"
DEFAULT_SYSTEM_PROMPT = "Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им."


def get_prompt(context: list[str], question: str) -> str:
    prompt = (f"Не повторяйся. Только русский язык. Не задавай вопросов. Контекст: {context}, Вопрос: {question}, Ответ:")
    n = len(context)

    while len(prompt) >= 2048:
        prompt = (f"Не повторяйся. Только русский язык. Не задавай вопросов. Контекст: {context[:n - 1]}, Вопрос: {question}, Ответ:")
        n -= 1

    return prompt


class Conversation:
    def __init__(
            self,
            message_template=DEFAULT_MESSAGE_TEMPLATE,
            system_prompt=DEFAULT_SYSTEM_PROMPT,
            response_template=DEFAULT_RESPONSE_TEMPLATE
    ):
        self.message_template = message_template
        self.response_template = response_template
        self.messages = [{
            "role": "system",
            "content": system_prompt
        }]

    def add_user_message(self, message):
        self.messages.append({
            "role": "user",
            "content": message
        })

    def add_bot_message(self, message):
        self.messages.append({
            "role": "bot",
            "content": message
        })

    def get_prompt(self, tokenizer):
        final_text = ""
        for message in self.messages:
            message_text = self.message_template.format(**message)
            final_text += message_text
        final_text += DEFAULT_RESPONSE_TEMPLATE
        return final_text.strip()


def generate(model, tokenizer, prompt, generation_config):
    data = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(
        **data,
        generation_config=generation_config
    )[0]
    output_ids = output_ids[len(data["input_ids"][0]):]
    output = tokenizer.decode(output_ids, skip_special_tokens=True)
    return output.strip()


config = PeftConfig.from_pretrained(LLM_MODEL_NAME)
llm_model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    torch_dtype=torch.float16,
    device_map=device
)
llm_model = PeftModel.from_pretrained(
    llm_model,
    LLM_MODEL_NAME,
    torch_dtype=torch.float16
)
llm_model.eval()

llm_tokenizer = AutoTokenizer.from_pretrained(LLM_MODEL_NAME, use_fast=False)
generation_config = GenerationConfig.from_pretrained(LLM_MODEL_NAME)
generation_config.temperature = 0.01
# generation_config.frequency_penalty = 1.3


# собственно вызов функции генерации ответа в ЛЛМ Сайга. Подается на вход запрос и контекст
def get_answer(question: str, context: list[str]):
    prompt = get_prompt(context, question)

    return generate(llm_model, llm_tokenizer, prompt, generation_config)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
# собранный вариант реализации
def pipeline(question: str, collection_name: str = "FULL_UPLOAD") -> str:
    context = get_context(question, collection_name)

    return get_answer(question, ",".join(context))

In [12]:
pipeline("На основании какого документа (не НПА) разрабатываются и устанавливаются нормативы допустимых выбросов загрязняющих веществ в атмосферный воздух?")

'Нормативы допустимых выбросов загрязнителей в атмосферный воздух устанавливаются на основе Постановлений Правительства Российской Федерации от 19.08.2004 № 653 «Об утверждении Правил технического регулирования по контролю за выбросами загрязняющих веществ в атмосферу», а также других нормативных правовых актов.'

## Чтение из файлов и обработка csv

In [13]:
test_df = pd.read_csv("test.csv", sep="\t")

In [14]:
test_df

,№ п/п,Вопрос,Ответ,Документ
0,1,"Объяснить, что такое источник выбросов, источн...",NaN,Нет
1,2,Указать этапы разработки проекта начиная с пол...,NaN,Нет
2,3,Расписать состав тома ПДВ,NaN,Нет
3,4,Как присваиваются номера источников выбросов п...,NaN,Нет
4,5,Что такое газоочистные установки? Приведите их...,NaN,Нет
...,...,...,...,...
87,88,"Каковы источники выбросов, имеющие произвольну...",NaN,"Книга 1 - Инвентаризация Эко Агро, Таблица 4.4"
88,89,Сильнее ли жидкие и газообразные загрязняющие ...,NaN,"Книга 1 - Инвентаризация Эко Агро, Таблица 4.8"
89,90,Какой годовой выброс в тоннах зерновой пыли?\n,NaN,"Книга 1 - Инвентаризация Эко Агро, Таблица 1.1.2"
90,91,Создаётся ли хлопковая пыль?\n,NaN,"Книга 1 - Инвентаризация Эко Агро, Таблица 1.1.2"


In [20]:
from csv import DictReader as CsvDictReader

answers = []

with open(DATASET_TASKS_FILE_PATH, mode="r") as dataset_tasks_file:
    dataset_tasks_file_reader = CsvDictReader(dataset_tasks_file, delimiter="\t")
    for task in dataset_tasks_file_reader:
        print(task)
        question = task["Вопрос"]
        document = task["Документ"][:7]
        
        if document == "Книга 1":
            collection_name = "Том 1 Инвентаризация Эко Агро"
        elif document == "Книга 2":
            collection_name = "Том 2 ПДВ Эко Агро"
        else:
            collection_name = "FULL_UPLOAD"
    
        context = get_context(question, collection_name)
        answer = get_answer(question, ",".join(context))
    
        print(answer)
        print("--------------------------")
        answers.append(answer)

{'№ п/п': '1', 'Вопрос': 'Объяснить, что такое источник выбросов, источник выделения.', 'Ответ': '', 'Документ': 'Нет'}
Источник выбросов – это место, где происходит выделение вредных веществ в атмосферу. Источником выделения являются различные технологические процессы, такие как производство электроэнергии, химические заводы, автомобильные заводы и т.д.
--------------------------
{'№ п/п': '2', 'Вопрос': 'Указать этапы разработки проекта начиная с получения в работу.', 'Ответ': '', 'Документ': 'Нет'}
1. Создание проекта. 2. Разработка технических требований. 3. Выбор технологии. 4. Разработка технологических процессов. 5. Разработка технологических процессов. 6. Разработка технологических процессов. 7. Разработка технологических процессов. 8. Разработка технологических процессов. 9. Разработка технологических процессов. 10. Разработка технологических процессов. 11. Разработка технологических процессов. 12. Разработка технологических процессов. 13. Разработка технологических процессов.

In [19]:
ans_copy = answers

In [21]:
len(answers)

92

In [22]:
ans_df = pd.DataFrame(range(1,93)
ans_df["answer"] = pd.DataFrame(answers)
ans_df.columns = []
№ п/п

,0
0,"Источник выбросов – это место, где происходит ..."
1,1. Создание проекта. 2. Разработка технических...
2,Состав тома ПДВ включает в себя следующие доку...
3,Номера источников выбросов присваиваются в соо...
4,Газоочистные установки - это специальные механ...
...,...
87,Из результатов расчета максимального разового ...
88,Жидкие и газообразные загрязнители могут быть ...
89,В годовой выбор можно было бы указать только о...
90,"Нет, создается не хлопковая пыль, а обычная пыль."


In [ ]:
ans_df